In [ ]:
install.packages("tidyverse")
library(tidyverse)

In [ ]:
# We need to define these as the first 1000 rows don't have any data so read_csv thinks they are logicals
col_specs <- cols( 
  sensor_value = col_character(),
  sensor_unit = col_character(),
  glatos_caught_date = col_date()
)

data <- read_csv("../data/detections.csv",  col_types = col_specs)
data

In [ ]:
dets_file <- file.path('../data', 'detections.csv')
rcv_file <- file.path('../data', 'deployments.csv')
tags_file <- file.path('../data', 'animal_tags.csv')

dets <- read_csv(dets_file, col_types = col_specs) #detections from acoustic receivers
Rxdeploy <- read_csv(rcv_file) #receiver station info
tags <- read_csv(tags_file) #tagged fish data

In [ ]:
head(dets)

In [ ]:
tail(dets)

In [ ]:
str(dets)

In [ ]:
dets[1:10,]

In [ ]:
head(Rxdeploy)

In [ ]:
head(tags)